In [1]:
import numpy as np
import tensorflow as tf
from pinn import get_network
from pinn.utils import connect_dist_grad
from glob import glob
from ase.collections import g2
from pinn.io import load_qm9, sparse_batch
from pinn.optimizers import get
import psutil
import os
import time
from pinn.layers import PolynomialBasis, GaussianBasis, ANNOutput
from pinn.networks.pinet import OutLayer, GCBlock, ResUpdate, PreprocessLayer
from pinn.utils import atomic_dress, get_atomic_dress
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from mendeleev import element
import gc
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import Callback

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
# Run this cell to disable GPU
# physical_devices = tf.config.list_physical_devices()
# tf.config.set_visible_devices(physical_devices[0], 'CPU')
# tf.config.set_visible_devices([], 'GPU')

In [3]:
batch_size = 256
epochs = 1

In [4]:
filelist = glob('/Users/miguelnavaharris/Project/QM9/*.xyz')
num_trainset_samples = 0.8 * len(filelist)
num_testset_samples = 0.2 * len(filelist)
dataset = load_qm9(filelist, splits={'train':8, 'test':2})
dress, error = get_atomic_dress(dataset['train'],[1,6,7,8,9])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-04-30 13:12:48.613113: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-30 13:12:48.613211: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-30 13:12:48.630701: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-04-30 13:12:48.630806: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [6]:
def get_traintest_sets(dataset,batch_size, buffer_size=20000):
    train_set = dataset['train'].cache().shuffle(buffer_size).apply(sparse_batch(batch_size))
    test_set = dataset['test'].cache().apply(sparse_batch(batch_size))
    return (train_set, test_set, batch_size)

def get_dataset_size(dataset):
    return len(list(dataset))
    
def preprocess_traintest_sets(train_set, test_set):
    for batch in train_set:
        batch = network.preprocess(batch)
        connect_dist_grad(batch)
    for batch in test_set:
        batch = network.preprocess(batch)
        connect_dist_grad(batch)

def get_compiled_network():
    optimizer = get(params['optimizer'])
    loss_fn = tf.keras.losses.mse
    network.compile(optimizer=optimizer, loss=loss_fn, metrics=[tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.MeanSquaredError()]) #setting run_eagerly=True was a possible fix for memory leak
    return network


In [7]:
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()


In [8]:
class PiNet(tf.keras.Model):
    """Keras model for the PiNet neural network

    Args:
        tensors: input data (nested tensor from dataset).
        atom_types (list): elements for the one-hot embedding.
        pp_nodes (list): number of nodes for pp layer.
        pi_nodes (list): number of nodes for pi layer.
        ii_nodes (list): number of nodes for ii layer.
        en_nodes (list): number of nodes for en layer.
        depth (int): number of interaction blocks.
        rc (float): cutoff radius.
        basis_type (string): type of basis function to use,
            can be "polynomial" or "gaussian".
        n_basis (int): number of basis functions to use.
        gamma (float or array): width of gaussian function for gaussian basis.
        center (float or array): center of gaussian function for gaussian basis.
        cutoff_type (string): cutoff function to use with the basis.
        act (string): activation function to use.
        preprocess (bool): whether to return the preprocessed tensor.
    """
    def __init__(self, atom_types=[1, 6, 7, 8, 9],  rc=4.0, cutoff_type='f1',
                 basis_type='polynomial', n_basis=4, gamma=3.0, center=None,
                 pp_nodes=[16, 16], pi_nodes=[16, 16], ii_nodes=[16, 16],
                 out_nodes=[16, 16], out_units=1, out_pool=False,
                 act='tanh', depth=4):

        super(PiNet, self).__init__()

        self.depth = depth
        self.preprocess = PreprocessLayer(atom_types, rc)
        self.activation = act

        if basis_type == 'polynomial':
            self.basis_fn = PolynomialBasis(cutoff_type, rc, n_basis)
        elif basis_type == 'gaussian':
            self.basis_fn = GaussianBasis(cutoff_type, rc, n_basis, gamma, center)

        self.res_update = [ResUpdate() for i in range(depth)]
        self.gc_blocks = [GCBlock([], pi_nodes, ii_nodes, activation=act)]
        self.gc_blocks += [GCBlock(pp_nodes, pi_nodes, ii_nodes, activation=act)
                           for i in range(depth-1)]
        self.out_layers = [OutLayer(out_nodes, out_units) for i in range(depth)]
        self.ann_output =  ANNOutput(out_pool)
        
    
    def train_step(self, tensors):

        e_data_original = tf.identity(tensors['e_data'])

        with tf.GradientTape() as tape:
            pred_transformed = self(tensors, training=True)

            if params['params']['e_dress']:
                dress = atomic_dress(tensors, params['params']['e_dress'], dtype=pred_transformed.dtype)
                e_data_transformed = tensors['e_data'] - dress
            else:
                e_data_transformed = tensors['e_data']

            # e_data_transformed *= params['params']['e_scale']
            loss = self.compiled_loss(e_data_transformed, pred_transformed, regularization_losses=self.losses)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Reverse the scaling and atomic dress for predictions and true atomic energies
        pred_original = pred_transformed #/ params['params']['e_scale']
        if params['params']['e_dress']:
            pred_original += dress

        # Compute the metrics on the original scale
        self.compiled_metrics.update_state(e_data_original, pred_original)

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, tensors):

        pred_transformed = self(tensors, training=False)
        e_data_original = tf.identity(tensors['e_data'])
        if params['params']['e_dress']:
            dress = atomic_dress(tensors, params['params']['e_dress'], dtype=pred_transformed.dtype)
            e_data_transformed = tensors['e_data'] - dress
        else:
            e_data_transformed = tensors['e_data'] 
        # e_data_transformed *= params['params']['e_scale']
        self.compiled_loss(e_data_transformed, pred_transformed, regularization_losses=self.losses)

        # Reverse the scaling and atomic dress for predictions and true atomic energies
        pred_original = pred_transformed #/ params['params']['e_scale']
        if params['params']['e_dress']:
            pred_original += dress

        # Compute the metrics on the original scale
        self.compiled_metrics.update_state(e_data_original, pred_original)

        return {m.name: m.result() for m in self.metrics}

    

    def call(self, tensors):
        tensors = self.preprocess(tensors)
        basis = self.basis_fn(tensors['dist'])[:, None, :]
        output = 0.0
        for i in range(self.depth):
            prop = self.gc_blocks[i]([tensors['ind_2'], tensors['prop'], basis])
            output = self.out_layers[i]([tensors['ind_1'], prop, output])
            tensors['prop'] = self.res_update[i]([tensors['prop'], prop])

        output = self.ann_output([tensors['ind_1'], output])
        ind = tensors['ind_1']
        nbatch = tf.reduce_max(ind)+1
        output_by_batch = tf.math.unsorted_segment_sum(output, ind[:, 0], nbatch)
        return output_by_batch

In [10]:
class MoleculesPerSec(tf.keras.callbacks.Callback):
    def __init__(self, no_batches, batch_size, logdir):
        self.no_batches = no_batches
        self.batch_size = batch_size
        self.no_molecules = self.no_batches * self.batch_size
        self.batch_number = 0
        self.writer = tf.summary.create_file_writer(logdir)  # Use logdir for creating a writer
        self.process = psutil.Process(os.getpid())  # Get the current process

    def on_train_batch_begin(self, batch, logs=None):
        self.batch_time_start = time.time()

    def on_train_batch_end(self, batch, logs=None):
        self.batch_number += 1
        batch_time = time.time() - self.batch_time_start
        molecules_per_second = self.batch_size / batch_time
        ram_usage_mb, swap_usage_mb = self.get_ram_and_swap_usage()

        step = self.model.optimizer.iterations.numpy()  # Get the current step from the model's optimizer

        with self.writer.as_default():
            tf.summary.scalar('batch_moleculespersec', molecules_per_second, step=step)
            tf.summary.scalar('batch_ram_usage_mb', ram_usage_mb, step=step)
            tf.summary.scalar('batch_swap_usage_mb', swap_usage_mb, step=step)

    def get_ram_and_swap_usage(self):
        memory_info = self.process.memory_info()  # Get memory info for the current process
        ram_usage_mb = memory_info.rss / (1024 * 1024)

        swap_info = psutil.swap_memory()
        swap_usage_mb = swap_info.used / (1024 * 1024)

        return ram_usage_mb, swap_usage_mb


In [11]:
params = {'optimizer': {
        'class_name': 'Adam',
        'config': {
            'learning_rate': {
                'class_name': 'ExponentialDecay',
                'config': {
                    'initial_learning_rate': 1e-3,
                    'decay_steps': 10000, 
                    'decay_rate': 0.994}}, 
                    'clipnorm': 0.01}},    
            'params': {
                  'learning_rate': 1e-3, # Relatively large learning rate
                  'e_scale': 627.5, # Here we scale the model to kcal/mol
                  'e_dress': dress
              }
          }

In [12]:
network = PiNet()

In [13]:
train_set, test_set, batch_size = get_traintest_sets(dataset, batch_size)
preprocess_traintest_sets(train_set, test_set)
no_batches = get_dataset_size(train_set)
steps_per_epoch = num_trainset_samples // batch_size
validation_steps = num_testset_samples // batch_size

In [15]:
network = get_compiled_network()
logdir = '/Users/miguelnavaharris/New_Benchmarks/Prediction_accuracy/M1/Rescaled_correctly/PiNet_20_epochs_256_dressnoscale/' +  str(batch_size)
# early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2, mode='min', restore_best_weights=True)
tb_callback = tf.keras.callbacks.TensorBoard(logdir, update_freq=1)
moleculespersec_callback = MoleculesPerSec(no_batches, batch_size, logdir)
callbacks=[tb_callback, moleculespersec_callback]# , early_stopping_callback]

2023-04-30 13:14:00.706035: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-04-30 13:14:00.706048: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2023-04-30 13:14:00.706475: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [16]:
network.fit(train_set, epochs=epochs,  validation_data=test_set, callbacks=callbacks)

Epoch 1/20
Shape mismatch in elems: Tensor("pi_net/preprocess_layer/cond/Shape:0", shape=(1,), dtype=int32)
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


/Users/miguelnavaharris/miniforge3/envs/pinn/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("concat_1:0", shape=(None,), dtype=int32), values=Tensor("concat:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/pi_net/gc_block_3/pi_layer_3/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users/miguelnavaharris/miniforge3/envs/pinn/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("concat_3:0", shape=(None,), dtype=int32), values=Tensor("concat_2:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/pi_net/gc_block_2/pi_layer_2/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users

Shape mismatch in elems: Tensor("pi_net/preprocess_layer/cond/Shape:0", shape=(1,), dtype=int32)


2023-04-30 13:14:03.614607: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


      1/Unknown - 3s 3s/step - loss: 6130.5352 - mean_absolute_error: 75.6426 - mean_squared_error: 6130.5352

...

      2/Unknown - 4s 524ms/step - loss: 3271.3008 - mean_absolute_error: 47.1252 - mean_squared_error: 3271.3008

2023-04-30 13:14:05.768157: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2023-04-30 13:14:05.774239: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2023-04-30 13:14:05.780269: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /Users/miguelnavaharris/New_Benchmarks/Prediction_accuracy/M1/Rescaled_correctly/PiNet_20_epochs_256_dressnoscale/256/train/plugins/profile/2023_04_30_13_14_05
2023-04-30 13:14:05.784365: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /Users/miguelnavaharris/New_Benchmarks/Prediction_accuracy/M1/Rescaled_correctly/PiNet_20_epochs_256_dressnoscale/256/train/plugins/profile/2023_04_30_13_14_05/ch-gouldmac7.ch.ic.ac.uk.trace.json.gz
2023-04-30 13:14:05.789339: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /Users/miguelnavaharris/New_Benchmarks/Prediction_accuracy/M1/Res

    419/Unknown - 380s 902ms/step - loss: 28.1270 - mean_absolute_error: 1.0892 - mean_squared_error: 28.0863Shape mismatch in elems: Tensor("pi_net/preprocess_layer/cond/Shape:0", shape=(1,), dtype=int32)


2023-04-30 13:20:22.881410: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


419/419 [==============================] - 430s 1s/step - loss: 28.1270 - mean_absolute_error: 1.0892 - mean_squared_error: 28.0863 - val_loss: 0.1818 - val_mean_absolute_error: 0.3677 - val_mean_squared_error: 0.1818
Epoch 2/20
419/419 [==============================] - 680s 2s/step - loss: 0.1119 - mean_absolute_error: 0.2752 - mean_squared_error: 0.1118 - val_loss: 0.0050 - val_mean_absolute_error: 0.0617 - val_mean_squared_error: 0.0050
Epoch 3/20
419/419 [==============================] - 950s 2s/step - loss: 0.0134 - mean_absolute_error: 0.1008 - mean_squared_error: 0.0134 - val_loss: 0.0290 - val_mean_absolute_error: 0.1621 - val_mean_squared_error: 0.0290
Epoch 4/20
419/419 [==============================] - 1227s 3s/step - loss: 0.0321 - mean_absolute_error: 0.1496 - mean_squared_error: 0.0321 - val_loss: 0.0066 - val_mean_absolute_error: 0.0659 - val_mean_squared_error: 0.0066
Epoch 5/20
419/419 [==============================] - 1684s 4s/step - loss: 0.0089 - mean_absolute_e

# Predictions

In [18]:
network = tf.keras.models.load_model('/Users/miguelnavaharris/New_Benchmarks/NVIDIA/PiNet_rescaled_correctly/157_epochs_dressnoscale')

In [19]:
all_pred_energies = []
all_true_energies = []
num_heavy_atoms = []
molecule_masses = []
for test_batch in test_set:
    pred_energies = network(test_batch, training=False)
    # pred_energies /= params['params']['e_scale']
    if params['params']['e_dress']:
        dress = atomic_dress(test_batch, params['params']['e_dress'], dtype=pred_energies.dtype)
        pred_energies += dress

    pred_energies = pred_energies.numpy()
    atoms = test_batch["elems"].numpy()
    true_energies = test_batch["e_data"].numpy()
    molecule_indices = test_batch["ind_1"].numpy()

    all_pred_energies.extend(pred_energies.tolist())
    all_true_energies.extend(true_energies.tolist())

    unique_elements = np.unique(atoms)
    element_masses = {int(elem_num): element(int(elem_num)).mass for elem_num in unique_elements}

    for idx in range(true_energies.shape[0]):
        molecule_atoms = atoms[molecule_indices[:, 0] == idx]
        num_heavy = np.sum(molecule_atoms > 1)
        num_heavy_atoms.append(num_heavy)

        # Calculate molecule mass using Mendeleev
        molecule_mass = sum(element_masses[int(elem_num)] for elem_num in molecule_atoms)
        molecule_masses.append(molecule_mass)

results = pd.DataFrame({
    'true_energy': all_true_energies,
    'pred_energy': all_pred_energies,
    'num_heavy_atoms': num_heavy_atoms,
    'molecule_mass': molecule_masses,
})


2023-04-30 10:31:05.798541: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [20]:
# Assuming your dataframe is named results
results['abs_diff'] = (results['true_energy'] - results['pred_energy']).abs()
mae = results['abs_diff'].mean()

print("Mean Absolute Error (MAE):", mae)


Mean Absolute Error (MAE): 0.0029345662437935014


In [21]:
results.to_csv('/Users/miguelnavaharris/New_Benchmarks/NVIDIA/PiNet_rescaled_correctly/157_epochs_dressnoscale/predvstrue.csv')

# ASE Calculator

In [23]:
def _generator(molecule):
        data = {'coord': molecule.positions,
                'ind_1': np.zeros([len(molecule), 1]),
                'elems': molecule.numbers}
        yield data

def predict_energy(molecule):
        '''Takes an ASE Atoms object and outputs PiNet's energy prediction'''
        dtype=tf.float32
        dtypes = {'coord': dtype, 'elems': tf.int32, 'ind_1': tf.int32}
        shapes = {'coord': [None, 3], 'elems': [None], 'ind_1': [None, 1]}

        pred_dataset = tf.data.Dataset.from_generator(lambda:_generator(molecule), dtypes, shapes)

        for molecule in pred_dataset:
                molecule = network.preprocess(molecule)
                pred = network(molecule, training=False)
                ind = molecule['ind_1']
                nbatch = tf.reduce_max(ind)+1
                energy_prediction = tf.math.unsorted_segment_sum(pred, ind[:, 0], nbatch)
                energy_prediction = energy_prediction / params['params']['e_scale']
                if params['params']['e_dress']:
                        energy_prediction += atomic_dress(molecule, params['params']['e_dress'], dtype=energy_prediction.dtype)
                energy_prediction_numpy = energy_prediction.numpy()[0]
        return energy_prediction_numpy

In [27]:
predict_energy(g2['C2H6'])

-79.78593